In [ ]:
%pip install pandas scikit-learn numpy google-cloud-bigquery hopsworks


In [ ]:
!pip install hsfs[python]

# Feature Pipeline
### Componente: Backfill data


In [1]:
import pandas as pd

In [2]:
demography_df = pd.read_csv("https://repo.hops.works/dev/davit/churn/demography.csv")
customer_info_df = pd.read_csv("https://repo.hops.works/dev/davit/churn/customer_info.csv")
subscriptions_df = pd.read_csv("https://repo.hops.works/dev/davit/churn/subscriptions.csv")

In [3]:
customer_info_df

,customerID,Contract,tenure,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Month-to-month,1,Electronic check,Yes,29.85,29.85,No
1,5575-GNVDE,One year,34,Mailed check,No,56.95,1889.5,No
2,3668-QPYBK,Month-to-month,2,Mailed check,Yes,53.85,108.15,Yes
3,7795-CFOCW,One year,45,Bank transfer (automatic),No,42.30,1840.75,No
4,9237-HQITU,Month-to-month,2,Electronic check,Yes,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...
7038,6840-RESVB,One year,24,Mailed check,Yes,84.80,1990.5,No
7039,2234-XADUH,One year,72,Credit card (automatic),Yes,103.20,7362.9,No
7040,4801-JZAZL,Month-to-month,11,Electronic check,Yes,29.60,346.45,No
7041,8361-LTMKD,Month-to-month,4,Mailed check,Yes,74.40,306.6,Yes


In [4]:
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
#Resolver problemas de nulos:

customer_info_df["TotalCharges"] = pd.to_numeric(customer_info_df["TotalCharges"], errors='coerce')
customer_info_df["TotalCharges"].fillna(0, inplace=True)

customer_info_df["Churn"].replace({"No" : 0, "Yes" : 1}, inplace=True)

In [7]:
#Creación de feature group:

import hopsworks
project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/61594


FeatureStoreException: Trying to instantiate Python as engine, but 'python' extras are missing in HSFS installation. Install with `pip install hsfs[python]`.

In [ ]:
#import hopsworks
import pandas as pd
import joblib
import os
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')
#from hsml.schema import Schema
#from hsml.model_schema import ModelSchema

 # 2. Preprocesamiento
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.feature_selection import SelectPercentile, chi2

 # 3. Entrenamiento
from sklearn.linear_model import LogisticRegression

  # 3. Evaluación
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import(
    accuracy_score, confusion_matrix, classification_report, 
    roc_auc_score, roc_curve, auc
)

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
 def convert_dt(): 
    df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')
    df['clienteidentifier'] = df['clienteidentifier'].astype(str)
    df['multiplelines'] = df['multiplelines'].astype(str)
    df['internetservice'] = df['internetservice'].astype(str)
    df['onlinesecurity'] = df['onlinesecurity'].astype(str)
    df['onlinebackup'] = df['onlinebackup'].astype(str)
    df['deviceprotection'] = df['deviceprotection'].astype(str)
    df['techsupport'] = df['techsupport'].astype(str)
    df['streamingtv'] = df['streamingtv'].astype(str)
    df['streamingmovies'] = df['streamingmovies'].astype(str)
    df['contract'] = df['contract'].astype(str)
    df['paymentmethod'] = df['paymentmethod'].astype(str)
    df['gender'] = df['gender'].astype(str)
    df['paperlessbilling'] = df['paperlessbilling'].astype(str)
    df['partner'] = df['partner'].astype(str)
    df['dependents'] = df['dependents'].astype(str)
    df['phoneservice'] = df['phoneservice'].astype(str)
    df['seniorcitizen'] = df['seniorcitizen'].astype(str)
    df['monthlycharges'] = df['monthlycharges'].astype(float)
    df['totalcharges'] = df['totalcharges'].astype(float)
    df['tenure'] = df['tenure'].astype(float)
    df['churn'] = df['churn'].astype(str)
    #df['fecha_ingreso'] = pd.to_datetime(df['fecha_ingreso']).dt.date
    
    new_order = ['clienteidentifier','multiplelines','internetservice','onlinesecurity','onlinebackup',
             'deviceprotection','techsupport','streamingtv','streamingmovies','contract','paymentmethod',
             'gender','paperlessbilling','partner','dependents','phoneservice','seniorcitizen',
             'monthlycharges','totalcharges','tenure','churn']
    
    df = df.reindex(columns=new_order)
    
    return df



In [ ]:
data = convert_dt()

In [ ]:
new_order = ['clienteidentifier','multiplelines','internetservice','onlinesecurity','onlinebackup',
             'deviceprotection','techsupport','streamingtv','streamingmovies','contract','paymentmethod',
             'gender','paperlessbilling','partner','dependents','phoneservice','seniorcitizen',
             'monthlycharges','totalcharges','tenure','churn'
]

df = data.reindex(columns=new_order)

In [ ]:
df = df.drop(['clienteidentifier'], axis = 1)

In [ ]:
df.dtypes

In [ ]:
df

In [ ]:
# Dividir las variables en numéricas y categóricas
#label = ['churn']
#numeric_features = ['tenure', 'monthlycharges', 'totalcharges']
#categorical_features = ['gender', 'seniorcitizen', 'partner', 'dependents', 'phoneservice', 'multipleLines',
 #                       'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
  #                      'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod']

In [ ]:
numeric_features = ['monthlycharges', 'totalcharges', 'tenure']

categorical_features = ['multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport', 
 'streamingtv', 'streamingmovies', 'contract', 'paymentmethod', 'gender', 'paperlessbilling', 'partner',
 'dependents', 'phoneservice', 'seniorcitizen']

In [ ]:

def remove_duplicates(df):
    df.drop_duplicates(inplace=True)
    return df

def impute_missing_values(df):
    # Variables categóricas
    categorical_cols = df.select_dtypes(include='object').columns
    #categorical_cols = categorical_cols.drop('churn')  # Excluir 'churn'
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    df[categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])

    # Variables numéricas
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    numeric_imputer = SimpleImputer(strategy='mean')
    df[numeric_cols] = numeric_imputer.fit_transform(df[numeric_cols])

    return df

def normalize_numeric_variables(df):
    numeric_cols = ['monthlycharges', 'totalcharges', 'tenure']

    scaler = MinMaxScaler()
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    return df

    
def perform_one_hot_encoding(df, categorical_features):
    # Crea el codificador OneHotEncoder
    encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    
    # Selecciona las variables categóricas indicadas
    categorical_col = df[categorical_features]
    
    # Ajusta el codificador OneHotEncoder con las categorías conocidas
    encoder.fit(categorical_col)
    
    # Transforma las variables categóricas en one-hot encoding
    encoded_data = encoder.transform(categorical_col)
    
    # Obtiene los nombres de las nuevas columnas generadas
    feature_names = encoder.get_feature_names_out(categorical_features)
    
    # Crea un DataFrame con las nuevas columnas one-hot encoding
    encoded_df = pd.DataFrame(encoded_data, columns=feature_names)
    
    # Combina el DataFrame codificado con el resto del DataFrame original
    df_encoded = pd.concat([df.drop(categorical_features, axis=1), encoded_df], axis=1)
    
    return df_encoded



In [ ]:
df = remove_duplicates(df)
df = impute_missing_values(df)
df = normalize_numeric_variables(df)
df = perform_one_hot_encoding

In [ ]:
df = remove_duplicates(df)

In [ ]:
df = impute_missing_values(df)

In [ ]:
null_counts = df.isnull().sum()
print(null_counts)

In [ ]:
df = normalize_numeric_variables(df)

In [ ]:
df

In [ ]:
df = perform_one_hot_encoding(df, categorical_features)

In [ ]:
df.dtypes

In [ ]:
df

In [ ]:
null_counts = df.isnull().sum()
print(null_counts)

### Feature Engineering:

##### Dataset: Train

In [ ]:
df["churn"] = df.churn.map({'Yes': 1, "No": 0})

In [ ]:
X = df.drop(["clienteidentifier","churn"], axis = 1)
y = df["clienteidentifier","churn"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def print_score(true, pred, train=True):
    if train:
        clf_report = pd.DataFrame(classification_report(true, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(true, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(true, pred)}\n")
        
    elif train==False:
        clf_report = pd.DataFrame(classification_report(true, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(true, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(true, pred)}\n")

In [ ]:
clf.fit(X_train, y_train.ravel())